# RFM用户分群RFM会员价值度模型案例

## RFM 如何实施
  - 要有消费的流水数据,包含用户ID，消费时间，消费金额用户ID分组,(计算RFM的时间每个用户消费时间的最大值)R
  - 用户ID分组统计一共有多少条消费记录 F
  - 用户ID分组,对消费金额求和 M

In [2]:
#导入模块
import time
import numpy as np
import pandas as pd
from pyecharts.charts import Bar3D
import warnings
warnings.filterwarnings('ignore')

In [3]:
#导入数据
sheet_names = ['2015','2016','2017','2018','会员等级']
sheet_datas = [pd.read_excel('datas/sales.xlsx',sheet_name=i) for i in sheet_names]
# 上面的代码也可以用下面的方式实现
# sheet_data_2015 =pd.read_excel('data/sales.xlsx', sheet_name='2015')
# sheet_data_2016 =pd.read_excel('data/sales.xlsx', sheet_name='2016')
# sheet_data_2017 =pd.read_excel('data/sales.xlsx', sheet_name='2017')
# sheet_data_2018 =pd.read_excel('data/sales.xlsx', sheet_name='2018')
# sheet_data_level =pd.read_excel('data/sales.xlsx', sheet_name='会员等级')
# sheet_datas = [sheet_data_2015, sheet_data_2016,sheet_data_2017, sheet_data_2018, sheet_data_level]

In [9]:
#查看数据基本情况
for each_name,each_data in zip(sheet_names,sheet_datas):
    print('[data summary for ============={}===============]'.format(each_name))
    print('Overview:','\n',each_data.head(4))# 展示数据前4条
    print('DESC:','\n',each_data.describe())# 数据描述性信息
    print('NArecords',each_data.isnull().any(axis=1).sum()) #缺失值记录数  
    print('Dtypes',each_data.dtypes) # 数据类型# 

[data summary for =============2015===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NArecords 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
[data summary for =============2016===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  39288120141  428202

In [7]:
#数据预处理
# sheet_datas是5个df组成的矩阵，sheet_datas[:-1]表示前4个df组成矩阵
# enumerate(list)返回下标和对应的元素
for index,each_data in enumerate(sheet_datas[:-1]):
    # new_sheet_datas[index] 表示 new_sheet_datas中下标为index的元素，在这里是一个dataframe对象
    sheet_datas[index] = each_data.dropna()# 丢弃缺失值记录
    sheet_datas[index] = each_data[each_data['订单金额'] > 1]# 丢弃订单金额<=1的记录
    # new_sheet_datas[index]['max_year_date'] 即dataframe对象中的max_year_date列
    sheet_datas[index]['max_year_date'] =each_data['提交日期'].max() # 增加一列最大日期值
sheet_datas # 查看sheet_datas

[              会员ID         订单号       提交日期    订单金额 max_year_date
 0      15278002468  3000304681 2015-01-01   499.0    2015-12-31
 1      39236378972  3000305791 2015-01-01  2588.0    2015-12-31
 2      38722039578  3000641787 2015-01-01   498.0    2015-12-31
 3      11049640063  3000798913 2015-01-01  1572.0    2015-12-31
 4      35038752292  3000821546 2015-01-01    10.1    2015-12-31
 ...            ...         ...        ...     ...           ...
 30769  39368100847  4281994827 2015-12-31   828.0    2015-12-31
 30770       409757  4282010457 2015-12-31   199.0    2015-12-31
 30771  38380526114  4282017675 2015-12-31   208.0    2015-12-31
 30772     28074988  4282019440 2015-12-31    89.0    2015-12-31
 30773  39460363230  4282025309 2015-12-31   719.0    2015-12-31
 
 [30574 rows x 5 columns],
               会员ID         订单号       提交日期     订单金额 max_year_date
 0      39288120141  4282025766 2016-01-01    76.00    2016-12-31
 1      39293812118  4282037929 2016-01-01  7599.00    2016

In [10]:
#将前四个dataframe联合为一个df，并添加间隔日期和所在年份两列
# 汇总所有数据, 使用pandas.concat连接前四个dataframe
data_merge = pd.concat(sheet_datas[:-1])
# 计算日期间隔天数（该订单日期距离当年最后1天的天数），并添加列（该列的数据类型为timedelta64[ns]）
data_merge['date_interval'] =data_merge['max_year_date']-data_merge['提交日期']
# 转换日期间隔为数字
data_merge['date_interval'] =data_merge['date_interval'].dt.days
# 获取该订单的年份，并添加列
data_merge['year'] = data_merge['提交日期'].dt.year
data_merge.head()

,会员ID,订单号,提交日期,订单金额,max_year_date,date_interval,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015


## RFM原始值计算

In [29]:
#分组后聚合计算
# 基于year、会员ID列做分组之后，分别对date_interval、提交日期、订单金额做不同的运算
# as_index=False表示重置索引
rfm_gb = data_merge.groupby(['year', '会员ID'],as_index=False).agg({
    # R 求分组后date_interval列中最小值：计算当年该会员最后一次订单距离年末12月31日的间隔天数
    'date_interval': 'min',
    # F 订单频率，计算当年该会员一共消费多少次，也可以对订单号列进行count计算
    '提交日期': 'count',
    # M 计算订单总金额：计算当年该会员一共消费多少钱
    '订单金额': 'sum'
})
# 重命名列名
rfm_gb.columns = ['年度', '会员ID', 'r', 'f', 'm']
rfm_gb.head()

,年度,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0


In [33]:
#查看数据分布
# rfm_gb.iloc[:,2:]表示只选择rfm_gb的rfm三类数据，并返回新df，再做.describe().T操作，查看数据分布情况
desc_pd = rfm_gb.iloc[:, 2:].describe().T
desc_pd
#通过观察数据分布情况，自定义区间边界
r_bins = [-1, 79, 255, 365]
#rfm_gb.f.quantile(0.90)  #计算分界值
f_bins = [0, 2, 5, 130] # f数据的分布比较极端，所以这里采用较小的值
m_bins = [0, 69, 1199, 206252]

1.0

## RFM计算过程

In [16]:
#RFM分箱得分
# 对rfm_gb['r']的值按照r_bins进行划分，划分结果对应为新的值，新的值为labels中的对应值
#pd.cut(列表，边界列表或者分区数量，赋值列表)
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins,labels=[i for i in range(len(r_bins)-1, 0, -1)]) #计算R得分
# range函数：range(start, stop[, step])
#print([i for i in range(len(r_bins)-1, 0, -1)])
rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins,labels=[i+1 for i in range(len(f_bins)-1)]) # 计算F得分
rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins,labels=[i+1 for i in range(len(m_bins)-1)]) # 计算M得分
rfm_gb.head()

[3, 2, 1]


,年度,会员ID,r,f,m,r_score,f_score,m_score
0,2015,267,197,2,105.0,2,1,2
1,2015,282,251,1,29.7,2,1,1
2,2015,283,340,1,5398.0,1,1,3
3,2015,343,300,1,118.0,1,1,2
4,2015,525,37,3,213.0,3,2,2


In [18]:
#计算RFM组合
# r_score、f_score、m_score三列转为str类型
rfm_gb['r_score'] =rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] =rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] =rfm_gb['m_score'].astype(np.str)
# r_score、f_score、m_score三列拼接字符串，并添加新列
# rfm_gb['rfm_group'] =rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(rfm_gb['m_score'])
rfm_gb['rfm_group'] = rfm_gb['r_score'] +rfm_gb['f_score'] + rfm_gb['m_score']
rfm_gb.head()

,年度,会员ID,r,f,m,r_score,f_score,m_score,rfm_group
0,2015,267,197,2,105.0,2,1,2,212
1,2015,282,251,1,29.7,2,1,1,211
2,2015,283,340,1,5398.0,1,1,3,113
3,2015,343,300,1,118.0,1,1,2,112
4,2015,525,37,3,213.0,3,2,2,322


In [ ]:
#保存RFM结果到Excel
rfm_gb.to_excel('sales_rfm_score1.xlsx') # 保存数据为Excel

##  RFM图形展示

In [22]:
# 图形数据汇总
display_data =rfm_gb.groupby(['rfm_group','年度'],as_index=False)['会员ID'].count()
display_data.columns =['rfm_group','年度','number']
display_data['rfm_group'] =display_data['rfm_group'].astype(np.int32)
display_data.head()

,rfm_group,年度,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811


In [1]:
# 显示图形
from pyecharts.commons.utils import JsCode
import pyecharts.options as opts
# 颜色池
range_color = ['#313695', '#4575b4', '#74add1','#abd9e9', '#e0f3f8', '#ffffbf','#fee090', '#fdae61', '#f46d43','#d73027', '#a50026']
range_max = int(display_data['number'].max())
c = (
    Bar3D()#设置了一个3D柱形图对象
    .add(
        "",#图例
        [d.tolist() for d in display_data.values],#数据
        xaxis3d_opts=opts.Axis3DOpts(type_="category",name='分组名称'),#x轴数据类型，名称，rfm_group
        yaxis3d_opts=opts.Axis3DOpts(type_="category",name='年份'),#y轴数据类型，名称，year
        zaxis3d_opts=opts.Axis3DOpts(type_="value",name='会员数量'),#z轴数据类型，名称，number
    ).set_global_opts( # 全局设置
                        visualmap_opts=opts.VisualMapOpts(max_=range_max,
                        range_color=range_color), #设置颜色，及不同取值对应的颜色
                        title_opts=opts.TitleOpts(title="RFM分组结果"),#设置标题
    )
)
c.render_notebook() #在notebook中显示

NameError: name 'display_data' is not defined

## 案例结论

In [24]:
#计算RFM分组会员数占比
display_data
# 按RFM_Group分组之后，对number求和，返回新的df
rfm_percent =display_data.groupby(['rfm_group'],as_index=False)['number'].sum()
# 计算当前RFM_Group分组中会员总数百分比，并添加列
rfm_percent['count_per'] = rfm_percent['number'] /rfm_percent['number'].sum()
# 转换为百分数，保留2位小数
rfm_percent['count_per'] =rfm_percent['count_per'].apply(lambda x: format(x,'.2%'))
# 按number列进行排序，由大到小
rfm_percent = rfm_percent.sort_values(by='number',ascending=False)
rfm_percent

,rfm_group,number,count_per
9,212,36839,24.79%
8,211,19023,12.80%
17,312,18655,12.55%
1,112,16846,11.34%
10,213,16370,11.02%
16,311,9274,6.24%
0,111,9118,6.14%
18,313,8341,5.61%
2,113,7535,5.07%
5,123,1932,1.30%
